In [2]:
%load_ext autoreload
%autoreload 2

import re
import os
import sys
sys.path.insert(0, os.path.realpath('/content/jupyter/mta-accessibility/analysis/train_crowding'))
from gcs_utils import gcs_util

import os.path
from os import listdir
from os.path import isfile, join
# from google.transit import gtfs_realtime_pb2 as gtfs_rt
# from protobuf_to_dict import protobuf_to_dict
from datetime import datetime
import numpy as np
import pandas as pd

pd.set_option('max_colwidth', 200)
gcs = gcs_util()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
%time

date_pattern = re.compile("realtime/vehicle_updates_([0-9]*)-([0-9]*)")
keep_dates = ['20200419','20200425','20200426']
loaded_files = []
# df = pd.DataFrame()

for blob in gcs.list_blobs('realtime'): 
    name = blob.name
    if bool(re.search("realtime/vehicle_updates_", name)):
        date = date_pattern.match(name)[1]
        time = date_pattern.match(name)[2]
        if date in keep_dates:
            loaded_files.append(blob.name)
            df_temp = gcs.read_dataframe(name)
            df = df.append(df_temp,sort=False).drop_duplicates()

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 8.11 µs


In [35]:
%timeit

date_pattern = re.compile("realtime/vehicle_updates_([0-9]*)-([0-9]*)")
keep_dates = ['20200423','20200424']
loaded_files = []
# df_wkday = pd.DataFrame()

for blob in gcs.list_blobs('realtime'): 
    name = blob.name
    if bool(re.search("realtime/vehicle_updates_", name)):
        date = date_pattern.match(name)[1]
        time = date_pattern.match(name)[2]
        if date in keep_dates:
            loaded_files.append(blob.name)
            df_temp = gcs.read_dataframe(name)
            df_wkday = df_wkday.append(df_temp,sort=False).drop_duplicates()

In [37]:
df.start_date.value_counts()

20200425    274562
20200418    232989
20200426    190199
20200419    160486
20200424     53172
20200417     50786
20200427        23
Name: start_date, dtype: int64

In [38]:
df_wkday.start_date.value_counts()

20200423    289122
20200422    286950
20200421    283024
20200420    268072
20200424    234822
20200419     38180
Name: start_date, dtype: int64

In [29]:
df.current_status.value_counts()

1.0    37258
2.0    30375
0.0    11215
Name: current_status, dtype: int64

In [40]:
df_wkday.head()

,alert.header_text.translation,current_status,current_stop_sequence,id,route_id,start_date,start_time,stop_id,stop_name,timestamp,trip_id,alert.informed_entity
0,NaN,2.0,38.0,000002,1,20200419,NaN,142S,South Ferry,2020-04-19 23:59:55,114550_1..S03R,NaN
1,NaN,0.0,34.0,000004,1,20200419,NaN,107N,215 St,2020-04-19 23:59:57,114950_1..N03R,NaN
2,NaN,0.0,28.0,000006,1,20200419,NaN,130S,23 St,2020-04-19 23:59:53,116050_1..S03R,NaN
3,NaN,1.0,25.0,000008,1,20200419,NaN,116N,125 St,2020-04-19 23:59:38,116450_1..N03R,NaN
4,NaN,2.0,18.0,000010,1,20200419,NaN,120S,96 St,2020-04-19 23:59:52,117550_1..S03R,NaN


In [39]:
df_wkend = df.copy()

In [41]:
df_wkend.head()

,alert.header_text.translation,current_status,current_stop_sequence,id,route_id,start_date,start_time,stop_id,stop_name,timestamp,trip_id,alert.informed_entity
0,NaN,2.0,36.0,000002,1,20200417,NaN,104N,231 St,2020-04-18 00:00:15,115100_1..N03R,NaN
1,NaN,2.0,36.0,000004,1,20200417,NaN,138S,Cortlandt St,2020-04-18 00:00:09,115150_1..S03R,NaN
2,NaN,0.0,30.0,000006,1,20200417,NaN,111N,181 St,2020-04-18 00:00:09,115900_1..N03R,NaN
3,NaN,1.0,30.0,000008,1,20200417,NaN,132S,14 St,2020-04-17 23:59:40,115950_1..S03R,NaN
4,NaN,1.0,29.0,000052,2,20200417,NaN,132S,14 St,2020-04-18 00:00:03,114900_2..S01R,NaN


In [52]:
df_wkday[df_wkday.stop_name.str.contains('Eastche')]

,current_status,current_stop_sequence,route_id,start_date,start_time,stop_id,stop_name,timestamp,trip_id
37,1.0,25.0,5,20200419,NaN,501N,Eastchester - Dyre Av,2020-04-19 23:59:30,114700_5..N73R
36,2.0,25.0,5,20200419,NaN,501N,Eastchester - Dyre Av,2020-04-20 00:31:36,117700_5..N73R
34,2.0,25.0,5,20200419,NaN,501N,Eastchester - Dyre Av,2020-04-20 00:32:59,117700_5..N73R
35,1.0,25.0,5,20200419,NaN,501N,Eastchester - Dyre Av,2020-04-20 00:33:30,117700_5..N73R
36,2.0,25.0,5,20200419,NaN,501N,Eastchester - Dyre Av,2020-04-20 01:00:25,120700_5..N73R
...,...,...,...,...,...,...,...,...,...
64,2.0,25.0,5,20200424,NaN,501N,Eastchester - Dyre Av,2020-04-24 23:40:55,112900_5..N75R
62,0.0,25.0,5,20200424,NaN,501N,Eastchester - Dyre Av,2020-04-24 23:41:44,112900_5..N75R
60,1.0,25.0,5,20200424,NaN,501N,Eastchester - Dyre Av,2020-04-24 23:42:02,112900_5..N75R
61,2.0,25.0,5,20200424,NaN,501N,Eastchester - Dyre Av,2020-04-24 23:55:46,114150_5..N75R


In [47]:
df_wkday.drop(['id','alert.header_text.translation','alert.informed_entity','id'],axis=1,inplace=True)
df_wkend.drop(['id','alert.header_text.translation','alert.informed_entity','id'],axis=1,inplace=True)

In [48]:
df_wkday = df_wkday.drop_duplicates()
df_wkend = df_wkend.drop_duplicates()

In [57]:
df_wkday2 = df_wkday[(df_wkday.current_status != 0) & (df_wkday.current_status != 2)]
df_wkend2= df_wkend[(df_wkend.current_status != 0) & (df_wkend.current_status == 2)]

In [ ]:
df_wkday.to_csv('data/weekday_gtfs_0420_0424.csv',index=False)

df_wkend.to_csv('data/wekend_gtfs_0418-0419_0425-0426.csv',index=False)

In [58]:
df_wkday2.to_csv('data/weekday_gtfs_0420_0424.csv',index=False)

df_wkend2.to_csv('data/wekend_gtfs_0418-0419_0425-0426.csv',index=False)